In [19]:
import pandas as pd
import numpy as np

In [20]:
# df= pd.read_csv('初步数据版.csv')
path = 'C:\\Users\\ASUS\\Desktop\\IOM魔改2\\IOM209\\初步数据版.csv'
try:
    # 将 low_memory 设置为 False
    df = pd.read_csv(path, low_memory=False)
    print("文件读取成功")
except FileNotFoundError:
    print("文件未找到，请检查文件路径。")
except Exception as e:
    print(f"发生其他错误: {e}")

文件读取成功


In [21]:
#遍历所有列，倘若列的数据类型是字符串（object 类型），就打印该列的列名和所有唯一值（unique()）
# TODO:有没有更好的方法？
for col in df.columns:
    if df[col].dtype == "object":
        print(f"{col}: {df[col].unique()}\n")

tradeTime: ['2016/8/9' '2016/7/28' '2016/12/11' ... '2012/1/28' '2011/1/30'
 '2009/1/11']

livingRoom: ['2' '3' '1' '4' '5' '6' '9' '7' '8' '0' '#NAME?']

drawingRoom: ['1' '2' '0' '4' '3' '5' '中 14' '中 15' '中 16' '中 6' '高 14' '顶 6' '低 6'
 '低 16' '高 12' '低 15' '高 6' '底 28' '底 11' '中 24' '底 20' '中 22']

bathRoom: ['1' '2' '3' '0' '4' '5' '6' '7' '2006' '2003' '1990' '2000' '1996' '2005'
 '2011' '未知' '1994' '2004']

floor: ['高 26' '高 22' '中 4' '底 21' '中 6' '中 8' '高 6' '高 10' '中 23' '底 11' '底 3'
 '高 24' '低 23' '中 19' '高 18' '低 25' '中 12' '中 14' '中 30' '中 27' '中 5'
 '低 18' '底 28' '中 11' '低 9' '顶 7' '顶 27' '低 6' '中 17' '顶 6' '中 24' '中 15'
 '底 5' '中 29' '顶 19' '顶 5' '中 9' '低 22' '顶 18' '低 16' '高 13' '高 9' '高 17'
 '底 6' '中 28' '低 26' '底 15' '高 16' '底 2' '低 7' '中 13' '低 33' '底 14' '高 15'
 '底 4' '顶 11' '中 32' '顶 16' '底 18' '顶 17' '低 14' '低 10' '底 20' '高 12'
 '低 31' '低 30' '低 19' '低 12' '中 10' '中 16' '顶 20' '底 19' '中 31' '低 13'
 '底 10' '高 25' '中 21' '中 20' '高 20' '低 21' '低 24' '顶 4' '高 21' '高 7'

In [22]:
#删除所有有缺失值的行
df = df.dropna()

In [23]:
# 打印数据集的形状（行数，列数）
print(df.shape)

(316438, 22)


In [24]:
#计算天安门的十进制经纬度
# 定义一个函数，将度分秒转换为十进制
def dms_to_decimal(d, m, s):
    #将度分秒转换为十进制
    return d + m / 60 + s / 3600

# 天安门的经纬度（度, 分, 秒）
latitude_dms = (39, 54, 26.37)  # 北纬
longitude_dms = (116, 23, 29.22)  # 东经

# 转换为十进制
latitude_decimal = dms_to_decimal(*latitude_dms)
longitude_decimal = dms_to_decimal(*longitude_dms)

# 打印天安门的十进制经纬度
print(f"The decimal coordinates of Tiananmen：({latitude_decimal}, {longitude_decimal})")

The decimal coordinates of Tiananmen：(39.907325, 116.39145)


In [25]:
#定义一个距离函数，即我的经纬度坐标距离天安门有多远，使无意义的经纬度变成具像化的距离
from math import radians, cos, sin, asin, sqrt
def distance(lat2, lon2,lat1=39.907325,lon1=116.39145):
     # 计算具体的经纬度差异
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    # 计算具体的每一行数据的经纬度差异
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

    c = 2 * asin(sqrt(a))
    r = 6371 # 地球半径/km
    return(c * r)

In [26]:
#删除原有数据集中值为“未知”“#NAME?”的数据，其中livingRoom、bathRoom、constructionTime存在
df = df[df['constructionTime'] != '未知']
df = df[df['bathRoom'] != '未知']
df = df[df['livingRoom'] != '#NAME?']
print(df.shape)

(297694, 22)


In [27]:
# 计算每个数据点与天安门的距离
df['distance'] = df.apply(lambda x: distance(x['Lat'], x['Lng']), axis=1)

In [28]:
#将constructionTime由字符串类型变成int类型
df['constructionTime'] = df['constructionTime'].astype(int)

In [29]:
#将tradeTime变成日期类型，然后再提取其year，然后给该列重新赋值，数据类型为int
# TODO df['tradeTime'] 列的数据类型可能不是日期时间格式，而是字符串或其他类型。pd.DatetimeIndex 无法直接解析非日期时间格式的数据，导致 year 属性无法访问。
df['tradeTime'] = pd.DatetimeIndex(df['tradeTime']).year

In [30]:
#计算买卖时的房龄
df['Age'] = df['tradeTime'] - df['constructionTime']

In [31]:
# 拆分“Floor”列的中文描述为 数字 和 中文楼层类型
lst_numeric = []
lst_str = []
for value in df['floor'].values:
    value = value.split()
    try:
        numeric = (value[1])
        string  = value[0]
    except:
        numeric = np.nan
        string = np.nan
    lst_numeric.append(numeric)
    lst_str.append(string)

In [32]:
#将高 中 低变成英文
lst_str_eng=[]
for string in lst_str:
    if string == '中':
        lst_str_eng.append(string.replace('中','middle'))
    elif string == '高':
        lst_str_eng.append(string.replace('高','high'))
    elif string == '底':
        lst_str_eng.append(string.replace('底','bottom'))
    elif string == '低':
        lst_str_eng.append(string.replace('低','low'))
    elif string == '未知':
        lst_str_eng.append(string.replace('未知','unknown'))
    elif string == '顶':
        lst_str_eng.append(string.replace('顶','top'))

In [33]:
# 创建新的DataFrame并将它们连接到原始DataFrame
df1 = pd.DataFrame(lst_str_eng,columns=['floorType'])
df2 = pd.DataFrame(lst_numeric,columns=['floorHeight'])
df = pd.concat([df,df1,df2],axis=1)

In [34]:
df

,Lng,Lat,tradeTime,followers,totalPrice,price,square,livingRoom,drawingRoom,kitchen,...,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage,distance,Age,floorType,floorHeight
0,116.475489,40.019520,2016.0,106.0,415.0,31680.0,131.00,2,1,1.0,...,0.217,1.0,0.0,1.0,7.0,56021.0,14.385302,11.0,high,26
1,116.453917,39.881534,2016.0,126.0,575.0,43436.0,132.38,2,2,1.0,...,0.667,1.0,1.0,0.0,7.0,71539.0,6.051819,12.0,high,22
2,116.561978,39.877145,2016.0,48.0,1030.0,52021.0,198.00,3,2,1.0,...,0.500,1.0,0.0,0.0,7.0,48160.0,14.930543,11.0,middle,4
3,116.438010,40.076114,2016.0,138.0,297.5,22202.0,134.00,3,1,1.0,...,0.273,1.0,0.0,0.0,6.0,51238.0,19.183033,8.0,bottom,21
4,116.428392,39.886229,2016.0,286.0,392.0,48396.0,81.00,2,1,1.0,...,0.333,0.0,1.0,1.0,1.0,62588.0,3.928671,56.0,middle,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bottom,6
297627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,middle,6
297651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,high,24
297656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,middle,24


In [35]:
#将floorType是“未知”的删除
df = df[df['floorType']!='unknown']

In [36]:
#删除“未知”的floorType
df = df[df['floorType'] != 'unknown']

In [37]:
df

,Lng,Lat,tradeTime,followers,totalPrice,price,square,livingRoom,drawingRoom,kitchen,...,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage,distance,Age,floorType,floorHeight
0,116.475489,40.019520,2016.0,106.0,415.0,31680.0,131.00,2,1,1.0,...,0.217,1.0,0.0,1.0,7.0,56021.0,14.385302,11.0,high,26
1,116.453917,39.881534,2016.0,126.0,575.0,43436.0,132.38,2,2,1.0,...,0.667,1.0,1.0,0.0,7.0,71539.0,6.051819,12.0,high,22
2,116.561978,39.877145,2016.0,48.0,1030.0,52021.0,198.00,3,2,1.0,...,0.500,1.0,0.0,0.0,7.0,48160.0,14.930543,11.0,middle,4
3,116.438010,40.076114,2016.0,138.0,297.5,22202.0,134.00,3,1,1.0,...,0.273,1.0,0.0,0.0,6.0,51238.0,19.183033,8.0,bottom,21
4,116.428392,39.886229,2016.0,286.0,392.0,48396.0,81.00,2,1,1.0,...,0.333,0.0,1.0,1.0,1.0,62588.0,3.928671,56.0,middle,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bottom,6
297627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,middle,6
297651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,high,24
297656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,middle,24


In [38]:
df['floorType'] = df['floorType'].replace('bottom', 1)
df['floorType'] = df['floorType'].replace('low', 2)
df['floorType'] = df['floorType'].replace('middle', 3)
df['floorType'] = df['floorType'].replace('high', 4)
df['floorType'] = df['floorType'].replace('top', 5)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_29596\3607642923.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['floorType'] = df['floorType'].replace('top', 5)


In [39]:
df

,Lng,Lat,tradeTime,followers,totalPrice,price,square,livingRoom,drawingRoom,kitchen,...,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage,distance,Age,floorType,floorHeight
0,116.475489,40.019520,2016.0,106.0,415.0,31680.0,131.00,2,1,1.0,...,0.217,1.0,0.0,1.0,7.0,56021.0,14.385302,11.0,4.0,26
1,116.453917,39.881534,2016.0,126.0,575.0,43436.0,132.38,2,2,1.0,...,0.667,1.0,1.0,0.0,7.0,71539.0,6.051819,12.0,4.0,22
2,116.561978,39.877145,2016.0,48.0,1030.0,52021.0,198.00,3,2,1.0,...,0.500,1.0,0.0,0.0,7.0,48160.0,14.930543,11.0,3.0,4
3,116.438010,40.076114,2016.0,138.0,297.5,22202.0,134.00,3,1,1.0,...,0.273,1.0,0.0,0.0,6.0,51238.0,19.183033,8.0,1.0,21
4,116.428392,39.886229,2016.0,286.0,392.0,48396.0,81.00,2,1,1.0,...,0.333,0.0,1.0,1.0,1.0,62588.0,3.928671,56.0,3.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6
297627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,6
297651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,24
297656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,24


In [40]:
# 删除所有包含NaN的行
df.dropna(inplace=True)

In [41]:
df['floorHeight'] = df['floorHeight'].astype(int)
df['livingRoom'] = df['livingRoom'].astype(int)
df['district'] = df['district'].astype(int)
df['tradeTime'] = df['tradeTime'].astype(int)
df['Age'] = df['Age'].astype(int)
df['renovationCondition'] = df['renovationCondition'].astype(int)
df['buildingStructure'] = df['buildingStructure'].astype(int)
df['elevator'] = df['elevator'].astype(int)
df['fiveYearsProperty'] = df['fiveYearsProperty'].astype(int)
df['subway'] = df['subway'].astype(int)
df['followers']  = df['followers'].astype(int)

In [42]:
# 重置索引
df.reset_index(inplace=True)

In [43]:
df

,index,Lng,Lat,tradeTime,followers,totalPrice,price,square,livingRoom,drawingRoom,...,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage,distance,Age,floorType,floorHeight
0,0,116.475489,40.019520,2016,106,415.0,31680.0,131.00,2,1,...,0.217,1,0,1,7,56021.0,14.385302,11,4.0,26
1,1,116.453917,39.881534,2016,126,575.0,43436.0,132.38,2,2,...,0.667,1,1,0,7,71539.0,6.051819,12,4.0,22
2,2,116.561978,39.877145,2016,48,1030.0,52021.0,198.00,3,2,...,0.500,1,0,0,7,48160.0,14.930543,11,3.0,4
3,3,116.438010,40.076114,2016,138,297.5,22202.0,134.00,3,1,...,0.273,1,0,0,6,51238.0,19.183033,8,1.0,21
4,4,116.428392,39.886229,2016,286,392.0,48396.0,81.00,2,1,...,0.333,0,1,1,1,62588.0,3.928671,56,3.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277424,297688,116.344294,39.889416,2012,0,168.0,29428.0,57.09,3,1,...,0.333,0,1,1,10,84021.0,4.488653,30,3.0,5
277425,297689,116.386269,39.888411,2012,0,234.0,32664.0,71.64,3,1,...,0.500,0,1,1,10,104198.0,2.149080,16,3.0,9
277426,297690,116.366087,39.894322,2012,0,349.0,34270.0,101.84,2,2,...,0.300,1,1,1,10,86359.0,2.602217,13,3.0,5
277427,297691,116.352899,39.889225,2012,0,144.0,27068.0,53.20,2,1,...,0.500,0,1,1,10,79570.0,3.855649,26,3.0,24


In [44]:
# 删除新生成的索引列
df.drop(['index'], axis=1, inplace=True)

In [45]:
df.to_csv("步骤二后的数据.csv",index=False)

In [46]:
print(df.shape)

(277429, 26)
